# Datalab II <a name='jump'></a>

    Groepsnummer: 2B
    Coach: Marjolijn de Jager
    Namen groepsleden: 
        - Yara Passchier, 22080600
        - Jesse Van Leeuwen, 22096337
        - Alyssa Leliveld, 22111026
        - Bart Jeninga, 22079793

# Table of contents

1. [Sprint 2](#sprint2)
    1. [Beschrijving tabellen](#beschrijftabellen)
        1. [Bepaling gevens van onze club](#gegevensclub)
        2. [Maken functie om gegevens te verzamelen](#functiebepaling)
            1. [Opstellen ranglijst van een seizoen](#ranglijst)
2. [Sprint 3](#sprint3)
    1. [Bepaal met behulp van SQL:](#SQL)
        1. [Toon het aantal wedstrijden dat jouw team heeft gespeeld per seizoen.](#wedstrijdenperseizoen)
        2. [Toon het aantal wedstrijden van jouw team in het kalenderjaar 2010 per seizoen.](#wedstrijden2010)
        3. [Toon hoeveel punten ieder team in jouw competitie gehaald heeft per seizoen.](#puntencompetitie)
        4. [Toon op welke plaats jouw team daarmee is geëindigd in de ranglijst.](#teamranglijst)
    2. [Bepalen invloed teameigenschappen.](#teaminfluence)
        1. [Het mergen van het aantal punten per team en de teameigenschappen.](#dataframeteameigenschappen)
        2. [Bepalen correlatie teameigenschappen en behaalde punten.](#correlatie)
        3. [Visueel maken bevindingen.](#visualisatie)

# Sprint 2 <a name="sprint2"></a>

In [1]:
import pandas as pd
import sqlite3
import warnings
warnings.filterwarnings('ignore')
con = sqlite3.connect("database.sqlite")

def show_query(query):
    """
       Deze functie print de output van je SQL query uit.
       Parameters:
                  query = Je SQL query.
                          Zorg ervoor dat deze parameter een string is.
    """
    query_output = pd.read_sql_query(query, con)
    return pd.DataFrame(query_output)


In [2]:
# Load the data into a DataFrame
df_country = show_query("SELECT * FROM Country")
df_league = show_query("SELECT * FROM League")
df_match = show_query("SELECT * FROM Match")
df_player = show_query("SELECT * FROM Player")
df_player_attributes = show_query("SELECT * FROM Player_Attributes")
df_team = show_query("SELECT * FROM Team")
df_team_attributes = show_query("SELECT * FROM Team_Attributes")

DatabaseError: Execution failed on sql 'SELECT * FROM Country': no such table: Country

# Beschrijving tabellen <a name="beschrijftabellen"></a>
Er zijn 7 tabellen:

- country: een tabel met landen. In deze tabel staan alle landen die mee doen met de voetbalwedstrijden.

- league: tabel met de leagues. Deze tabel bevat de data van welke competities er worden gespeeld.

- match: tabel van verschillende wedstrijden. De tabel match bevat informatie over de wedstrijden.

- player: tabel van alle spelers. In deze tabel kan alle informatie over spelers worden gevonden.

- player_attributes: tabel met de eigenschappen van de spelers. De tabel player_attributes bevat informatie over de kwaliteiten en eigenschappen van de spelers.

- team: tabel van de teams. In deze tabel staat alle informatie over de verschillende teams.

- team_attributes: tabel met de eigenschappen van de teams. In deze tabel staat alle informatie van de verschillende teams.

- team_attributes: tabel met de eigenschappen van de teams. En als laatste in deze tabel staan alle kwaliteiten en eigenschappen van de teams.

## 1. Bepaal de volgende gegevens van jouw club: <a name="gegevensclub"></a>
- team naam
- team id
- competitie id
- competitie naam
- De gegevens van alle wedstrijden die jouw club heeft gespeeld.

De eerste stap is het achterhalen van ons team id, SC Heerenveen.

In [ ]:
#bepaal team naam, team id, competitie id, competitie naam van SC Heerenveen
query = """

SELECT *

    FROM team

    WHERE team_long_name = 'SC Heerenveen'

"""

show_query(query)

Dit team_api_id gebruiken wij vervolgens alle informatie per wedstrijd thuis en uit gespeeld op te halen.

In [ ]:
#Bepaal gegevens alle wedstrijden SC Heerenveen
query = """

SELECT *

    FROM match 

    WHERE home_team_api_id = 10228
    OR away_team_api_id = 10228

"""

show_query(query)

[Terug naar het begin](#jump)

## 2. Maak een functie waarmee je per seizoen voor een gekozen club o.a. het volgende kunt bepalen: <a name="functiebepaling"></a>

- Aantal wedstrijden in dat seizoen.
- Aantal punten behaald in dat seizoen.
- Gemiddeld aantal punten per wedstrijd.
- Winst-, verlies- en gelijk percentage in uitwedstrijden en thuiswedstrijden.

Hier wordt de functie gemaakt om met 3 parameters direct alle gevraagde informatie terug te krijgen. In de docstring van de functie is te lezen hoe de functie werkt, welke parameters er gebruikt worden en wat er uit de functie komt.

In [ ]:
def matchinfo(team_id, season, league_id):
  
    """Bereken de wedstrijdresultaten van een gekozen team voor een gekozen seizoen.
    Parameters:
    -----------
    team_id : int
        Het ID-nummer van het team die je als parameter invult en de resultaten voor worden berekend.
    season : str
        Het seizoen die je als parameter invult waarvan de resultaten worden berekend.
    league_id: int
        Het ID-nummer van de competitie waarin de club speelt en op gefilterd moet worden.
  
    Returns:
    -----------
    pandas.DataFrame
        Een Pandas DataFrame met de wedstrijdresultaten van het team en het seizoen dat je hebt opgegeven bij de parameters. dit zijn de kolommen die het dataframe bevat:
        - totaal_games: het totale aantal van gespeelde wedstrijden in het gekozen seizoen
        - total_points: het totale aantal gehaalde punten in het gekozen seizoen
        - season: seizoen waar de rest van de informatie over gaat
        - team_id: het ID-nummer van het gekozen team
        - average_points: het gemiddelde aantal punten
        - perc_win_home: percentage van de gewonnen thuis wedstrijden
        - perc_draw_home: percentage van gelijk gespeelde thuis wedstrijden
        - perc_loss_home: percentage van de verloren thuis wedstrijden
        - perc_win_away: percentage van de gewonnen uit wedstrijden
        - perc_draw_away: percentage van gelijk gespeelde uit wedstrijden
        - perc_loss_away:  percentage van de verloren thuis wedstrijden
      
    """
    #finding team in home match
    home_match_results = []
    home_team_points = []
    home_df = df_match[(df_match['home_team_api_id'] == team_id) & (df_match['season'] == season) & (df_match['league_id'] == league_id)]
    for index,row in home_df.iterrows():
        if row['home_team_goal'] > row['away_team_goal']:
            home_match_results.append('win')
            home_team_points.append(3)
        elif row['home_team_goal'] == row['away_team_goal']:
            home_match_results.append('draw')
            home_team_points.append(1)
        else:
            home_match_results.append('loss')
            home_team_points.append(0)
 
 
    home_df['match_results'] = home_match_results
    home_df['team_points'] = home_team_points
 
    #finding team in away match
    away_match_results = []
    away_team_points = []
    away_df = df_match[(df_match['away_team_api_id'] == team_id) & (df_match['season'] == season) & (df_match['league_id'] == league_id)]
    for index,row in away_df.iterrows():
        if row['away_team_goal'] > row['home_team_goal']:
            away_match_results.append('win')
            away_team_points.append(3)
        elif row['away_team_goal'] == row['home_team_goal']:
            away_match_results.append('draw')
            away_team_points.append(1)
        else:
            away_match_results.append('loss')
            away_team_points.append(0)
 
    away_df['match_results'] = away_match_results
    away_df['team_points'] = away_team_points
 
    #sum of matches
    home_points = home_df['team_points'].sum()
    away_points = away_df['team_points'].sum()
  
    total_points = home_points + away_points
    total_match = len(home_df) + len(away_df)
    average_points = total_points / total_match
  
    #the percentages for home
    total_match_home = len(home_df)
    if total_match_home > 0:
        perc_win_home = len(home_df[home_df['team_points'] == 3]) / total_match_home * 100
        perc_draw_home = len(home_df[home_df['team_points'] == 1]) / total_match_home * 100
        perc_loss_home = len(home_df[home_df['team_points'] == 0]) / total_match_home * 100
    else:
        perc_win_home = 0
        perc_draw_home = 0
        perc_loss_home = 0
  
    #percentages for away
    total_match_away = len(away_df)
    if total_match_away > 0:
        perc_win_away = len(away_df[away_df['team_points'] == 3]) / total_match_away * 100
        perc_draw_away = len(away_df[away_df['team_points'] == 1]) / total_match_away * 100
        perc_loss_away = len(away_df[away_df['team_points'] == 0]) / total_match_away * 100
    else:
        perc_win_away = 0
        perc_draw_away = 0
        perc_loss_away = 0
  
    #creating the dataframe
    df = pd.DataFrame()
    df['season'] = [season]
    df['team_id'] = team_id
    df['league_id'] = league_id
    df['total_games'] = total_match
    df['total_points'] = total_points
    df['average_points'] = average_points
    df['perc_win_home'] = perc_win_home
    df['perc_draw_home'] = perc_draw_home
    df['perc_loss_home'] = perc_loss_home
    df['perc_win_away'] = perc_win_away
    df['perc_draw_away'] = perc_draw_away
    df['perc_loss_away'] = perc_loss_away
  
    return df

### Pas deze functie toe op een seizoen naar keuze en bepaal de ranglijst van dat seizoen. <a name="ranglijst"></a>

Vervolgens gebruiken wij deze functie om het toe te passen in een loop voor elk team, dit geeft ons een dataframe met de informatie van alle teams en hebben deze gerankschikt.

In [ ]:
# Toepassing ranglijst op alle teams voor seizoen 2008/2009

team_list = list(df_match['home_team_api_id'].unique())

df_season = pd.DataFrame()
for team in team_list:
    team_season_results = matchinfo (team, '2008/2009', 13274)
    df_season = pd.concat([df_season,team_season_results])

ranglijst = df_season.sort_values(by = ['total_points'], ascending = False)

In [ ]:
ranglijst = ranglijst.dropna().reset_index().set_index('team_id')
del ranglijst['index']
display(ranglijst)

[Terug naar het begin](#jump)

# Sprint 3 <a name="sprint3"></a>

## 1. Bepaal met behulp van SQL in deze sprint het volgende:  <a name="SQL"></a>

- Toon het aantal wedstrijden dat jouw team heeft gespeeld per seizoen.
- Toon het aantal wedstrijden van jouw team in het kalenderjaar 2010 per seizoen.
- Toon hoeveel punten ieder team in jouw competitie gehaald heeft per seizoen.
- Toon op welke plaats jouw team daarmee is geëindigd in de ranglijst.

### Toon het aantal wedstrijden dat jouw team heeft gespeeld per seizoen. <a name="wedstrijdenperseizoen"></a>

hier bekijken we hoeveel wedstrijden ons gekozen team in totaal heeft gespeeld in elk seizoen

In [ ]:
#Aantal wedstrijden HC Heerenveen per seizoen.
def wedstrijden(team):
    """ telt het aantal wedstrijden van het gekozen team in de parameter per seizoen
    parameters:
    -----------
    team: int
    de ID van het team waarvan het totaal aantal wedstrijden wordt geteld per seizoen
    
    returns:
    -----------
    pandas.DataFrame
    Een pandas DataFrame waarin je het totaal aantal gespeelde wedstrijden kan zien van het gekozen team in de parameter per seizoen.
    Dat laat de volgende kolommen zien:
    -season: het seizoen waarvan het aantal wedstrijden wordt geteld.
    -aantal_wedstrijden: het aantal wedstrijden gespeeld door het team dat is gekozen in de parameter per seizoen.
    
    """
    query = f"""
    SELECT season 
         , COUNT(match_api_id) AS aantal_wedstrijden
    FROM Match
    WHERE home_team_api_id = {team}
          OR away_team_api_id = {team}
    GROUP BY season;
    """
    return(show_query(query))

wedstrijden(10228)

### Toon het aantal wedstrijden van jouw team in het kalenderjaar 2010 per seizoen. <a name="wedstrijden2010"></a>

vervolgens bekijken we hier hoeveel wedstrijden ons team in totaal heeft gespeeld per seizoen in het jaar 2010

In [ ]:
#Aantal wedstrijden 2010 per seizoen HC Heerenveen
def seizoenwedstrijd(team):
    """ telt het aantal wedstrijden dat een bepaald team heeft gespeeld( in dit geval SC Heerenveen) in een bepaald jaar(in dit geval in het jaar 2010). 
    parameters:
    -----------
    team: int
    de ID van het team waarvan het totaal aantal wedstrijden word berekend
            
    returns:
    -----------
    pandas.DataFrame
    een pandas DataFrame waarin je het totaal aantal gespeelde wedstrijden, zowel de uit als thuiswedstrijden ziet van het team waarvan je het team_ID invoert in de parameter.
    Dat laat de volgende kolommen zien:
    -season: de seizoenen waarin de wedtsrijden zijn gespeeld die het jaar 2010 bevatten.
    -aantal_wedstrijden: het totaal aantal gespeelde wedstrijden van het gegeven seizoen en team.
       
    """ 
    query = f"""
    SELECT season
         , COUNT(match_api_id) AS aantal_wedstrijden
    FROM Match
    WHERE (home_team_api_id = {team}
          OR away_team_api_id = {team})
          AND date LIKE '2010%'
    GROUP BY season;
    """
    return(show_query(query))

seizoenwedstrijd(10228)

### Toon hoeveel punten ieder team in jouw competitie gehaald heeft per seizoen. <a name="puntencompetitie"></a>

hier berekenen we hoeveel punten elk team in de competitie van ons team heft behaald per seizoen dat ze hebben gespeeld

In [ ]:
#Punten per seizoen per team
def seizoenspunten(league_id, season):
    """ Laat het aantal punten van het gekozen team en seizoen zien die in de parameters zijn gegeven.
    parameters:
    -----------
    league_id: int
        De ID's van de verschillende leagues waar uit gekozen kan worden
    season: str
        het seizoen waarvan het aantal punten van het gekozen team wordt berekend
    
    returns:
    -----------
    pandas.DataFrame
    een pandas DataFrame die het aantal behaalde punten voor elk team voor de league en het seizoen die in de parameters worden gekozen laat zien.
    dat laat de volgende kolommen zien:
    team_name: de naam van het team 
    points: het aantal punten van het team in de gekozen league en seizoen
    
    """
 
    query = f"""
        SELECT team_long_name AS team_name
             , SUM(CASE WHEN home_team_api_id = team_api_id THEN
                         CASE WHEN home_team_goal > away_team_goal THEN + 3
                              WHEN home_team_goal = away_team_goal THEN + 1
                              ELSE + 0 END
                        WHEN away_team_api_id = team_api_id THEN
                         CASE WHEN away_team_goal > home_team_goal THEN + 3
                              WHEN away_team_goal = home_team_goal THEN + 1
                              ELSE + 0 END
                        END) AS points
    
    FROM Match
    JOIN Team
        ON home_team_api_id = team_api_id OR away_team_api_id = team_api_id

    WHERE season = "{season}" AND league_id = "{league_id}"
    GROUP BY team_name
    ORDER BY points DESC;


"""
    return(show_query(query))
seizoenspunten(13274, '2010/2011')

### Toon op welke plaats jouw team daarmee is geëindigd in de ranglijst. <a name="teamranglijst"></a>

vervolgens hebben we met de info van de vorige vraag een ranglijst kunnen maken om in te kunnen zien op welke plek alle teams uit de competitie zijn geëindigd. 

In [ ]:
#Plaats in ranglijst HC Heerenveen
def seizoenspunten(league_id, season):
    """ laat zien op welke plaats teams zijn geëindigd in een bepaalde league en seizoen die zijn gegeven in de parameters.
    parameters: 
    -----------
    league_id: int
        de ID van de league waarvan je de ranglijst van de teams van wil hebben.
    season: str
        het seizoen waarvan je de ranglijst van de teams van wil hebben.
        
        
    returns:
    -----------
    pandas.DataFrame
    een pandas DataFrame die de geëindigd plaats, teamnaam en bijbehorende punten laat zien.
    dat laat de volgende kolommen zien:
    rank: de geëindigde plaatsen van de teams van de gekozen league en seizoen.
    team_name: de namen van de teams van de gekozen league en seizoen.
    points: het aantal behaalde punten van de teams van de gekozen league en seizoen.

    
    """
    
    
    query = f"""
        SELECT ROW_NUMBER() OVER(ORDER BY SUM(CASE WHEN home_team_api_id = team_api_id THEN
                                  CASE WHEN home_team_goal > away_team_goal THEN + 3
                                      WHEN home_team_goal = away_team_goal THEN + 1
                                      ELSE + 0 END
                                 WHEN away_team_api_id = team_api_id THEN
                                  CASE WHEN away_team_goal > home_team_goal THEN + 3
                                        WHEN away_team_goal = home_team_goal THEN + 1
                                      ELSE + 0 END
                                 END)  DESC) AS rank
             , team_long_name AS team_name
             , SUM(CASE WHEN home_team_api_id = team_api_id THEN
                         CASE WHEN home_team_goal > away_team_goal THEN + 3
                              WHEN home_team_goal = away_team_goal THEN + 1
                              ELSE + 0 END
                        WHEN away_team_api_id = team_api_id THEN
                         CASE WHEN away_team_goal > home_team_goal THEN + 3
                              WHEN away_team_goal = home_team_goal THEN + 1
                              ELSE + 0 END
                        END) AS points
    
    FROM Match
    JOIN Team
        ON home_team_api_id = team_api_id OR away_team_api_id = team_api_id

    WHERE season = "{season}" AND league_id = "{league_id}"
    GROUP BY team_name
    ORDER BY points DESC;


"""
    return(show_query(query))
seizoenspunten(13274, '2010/2011')

## 2. Bepalen invloed teameigenschappen. <a name="teaminfluence"></a>

### Het mergen van het aantal punten per team en de teameigenschappen.  <a name="dataframeteameigenschappen"></a>

In [ ]:
df_punten = seizoenspunten(13274, '2008/2009')

def df_merge(df):
    """ voegt het aantal punten per gekozen team toe aan de teameigenschappen van het gekozen team.
    parameters:
    -----------
    df: pandas.DataFrame
        een dataframe met de teamnaam met hun bijbehorende punten in een gekozen seizoen
        
    returns:
    -----------
    pandas.DataFrame
    een dataframe die gemerged die team_api_id,team_name,points bevat met hun bijbehoordende team attributen bevat.
    dat laat de volgende kolommen zien:
    - team_api_id: de ID van het team.
    - team_name: de naam van het team.
    - points: het totaal aantal punten van het gekozen team in een gekozen seizoen.
    - team attributes: de attributen van de teams.
    
    """
    #merging on the team
    df_merged_t = df.merge(df_team, left_on='team_name', right_on='team_long_name')
    
    #groupby on the columns we need  
    df_sort = df_merged_t[["team_api_id","team_name", "points"]]

    #merging on tean attributes
    df_season_points = df_sort.merge(df_team_attributes, left_on='team_api_id', right_on='team_api_id')
    
    df_season_points["date"] = pd.to_datetime(df_season_points.date)
    df2 = df_season_points[df_season_points["date"].isin(pd.date_range("2010-08-01", "2011-06-01"))]
    
    return df2

In [ ]:
total_df = df_merge(df_punten)
total_df

### Bepalen correlatie teameigenschappen en behaalde punten.  <a name="correlatie"></a>

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

for team in total_df['team_name'].unique():

    df_corr = total_df.corr() 
    points_corr = df_corr["points"] 
    df_points_corr = pd.DataFrame(points_corr, columns=['points']) 
    df_heatmap = df_points_corr.drop(index='points')

    pd.set_option("expand_frame_repr", False)
    df_heatmap.dropna(inplace=True)
    plt.figure(figsize=(5,5))

    s = sns.heatmap(df_heatmap.sort_values(by=['points'], ascending=False), cmap='RdYlGn') 
    plt.title(f"Correlaties op het totaal aantal punten van {team}") 
    plt.show() 

In [ ]:
sns.regplot(data=total_df, x="points", y="defencePressure")
plt.title("Scatterplot van punten tegenover defencePressure") 
plt.show

In [ ]:
sns.regplot(data=total_df, x="points", y="buildUpPlayPassing")
plt.title("Scatterplot van punten tegenover buildUpPlayPassing") 
plt.show

### Visueel maken bevindingen.   <a name="visualisatie"></a>

[Terug naar het begin](#jump)